# 📊 Repositório de Consultas SQL para Analistas de Dados: **Departamento Fraud Prevention**
O departamento de Fraud Prevention é responsável por identificar, prevenir e mitigar atividades fraudulentas dentro de uma organização. Isso inclui o desenvolvimento e implementação de políticas, procedimentos e sistemas para detectar e evitar fraudes em transações financeiras, identidade, segurança cibernética e outros aspectos relacionados.

***
-  🔍 Contribuições: Estamos abertos a contribuições da comunidade de analistas de dados! Se você tiver consultas SQL úteis que gostaria de compartilhar ou sugestões para melhorias, sinta-se à vontade para enviar um pull request. Juntos, podemos expandir e aprimorar este repositório para beneficiar toda a comunidade de análise de dados.
- 🎯 Sinta-se à vontade para explorar, utilizar e contribuir para este repositório, e que ele possa ser uma ferramenta valiosa em sua jornada como analista de dados!

# 📌 **Identificação de Compras Acima da Média Móvel**

A identificação de compras acima da média móvel é uma técnica que ajuda a detectar transações que são anormalmente altas em relação ao comportamento histórico de compras de um usuário. Esta técnica é essencial para o departamento de prevenção de fraudes, pois permite identificar rapidamente possíveis atividades fraudulentas baseadas em um aumento súbito e significativo nos gastos.

#### **Importância da Segmentação para o Departamento de Prevenção de Fraudes**

A segmentação é crucial para a prevenção de fraudes, pois permite o agrupamento de transações e comportamentos de usuários em categorias distintas. Isso facilita a análise detalhada e a aplicação de critérios específicos para identificar fraudes. Com a segmentação, é possível diferenciar comportamentos normais e anômalos, ajustando os parâmetros de detecção de fraude conforme o perfil de cada segmento. Isso resulta em uma detecção mais precisa e uma redução nos falsos positivos, otimizando os recursos e o tempo do departamento de prevenção de fraudes.

#### **Explicação da Query**

A query a seguir calcula a média móvel de 30 dias das compras de cada usuário e identifica transações que excedem 30% acima dessa média.

```sql
WITH MovingAverage AS (
    SELECT
        user_id,
        purchase_date,
        amount,
        AVG(amount) OVER (PARTITION BY user_id ORDER BY purchase_date ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS moving_avg
    FROM
        purchases
)
SELECT
    user_id,
    purchase_date,
    amount,
    moving_avg,
    (CASE
        WHEN amount > 1.3 * moving_avg THEN 'Possivel Fraude'
        ELSE 'Normal'
    END) AS status
FROM
    MovingAverage;
```

#### Componentes da Query:

1. **CTE (Common Table Expression) `MovingAverage`:**
   - Calcula a média móvel de 30 dias para cada usuário.
   - Usa a função `AVG` com a cláusula `OVER` para calcular a média dos últimos 30 dias (29 dias anteriores + o dia atual) para cada transação.

2. **Consulta Principal:**
   - Seleciona o `user_id`, `purchase_date`, `amount` e a `moving_avg` calculada.
   - Compara o valor da compra (`amount`) com `130%` da média móvel (`1.3 * moving_avg`).
   - Classifica a transação como 'Possivel Fraude' se o valor for superior a esse limite, caso contrário, classifica como 'Normal'.

#### Benefícios:

- **Detecção Rápida:** Identifica transações anômalas em tempo real.
- **Personalização:** Adapta-se ao comportamento individual de cada usuário, oferecendo uma análise mais precisa.
- **Redução de Falsos Positivos:** Ao usar a média móvel, reduz a probabilidade de classificar transações legítimas como fraudulentas apenas devido a variações normais.

Esta técnica é uma ferramenta poderosa para o departamento de prevenção de fraudes, ajudando a proteger a organização contra perdas financeiras e a manter a confiança dos clientes.

# 📌 **Múltiplas Transações em Curto Período**

A detecção de múltiplas transações em um curto período é uma técnica que visa identificar padrões de comportamento anômalos, onde um usuário realiza um grande número de transações em um intervalo de tempo muito pequeno. Esta técnica é essencial para o departamento de prevenção de fraudes, pois pode sinalizar atividades fraudulentas, como o uso indevido de cartões de crédito ou contas comprometidas, onde um fraudador tenta maximizar os ganhos antes que a atividade seja detectada e bloqueada.

#### **Importância da Segmentação para o Departamento de Prevenção de Fraudes**

A segmentação é crucial para a prevenção de fraudes, pois permite categorizar os usuários e suas transações em diferentes grupos com base em comportamentos e características específicas. Isso facilita a criação de regras de detecção de fraudes mais precisas e adequadas para cada segmento. Com a segmentação, é possível definir limites e parâmetros de detecção ajustados conforme o perfil de cada grupo, aumentando a eficácia na identificação de fraudes e reduzindo o número de falsos positivos.

### **Implementação da Query com Segmentação**

A query a seguir identifica usuários que fizeram mais de um número específico de transações em um curto período de tempo, ajustando os limites de detecção para diferentes segmentos de usuários, como clientes VIP e usuários regulares.

```sql
WITH RecentTransactions AS (
    SELECT
        user_id,
        purchase_date,
        amount,
        user_type
    FROM
        purchases
    WHERE
        purchase_date >= NOW() - INTERVAL '10 minutes'
),

TransactionCounts AS (
    SELECT
        user_id,
        user_type,
        COUNT(*) AS transaction_count,
        MIN(purchase_date) AS first_purchase,
        MAX(purchase_date) AS last_purchase
    FROM
        RecentTransactions
    GROUP BY
        user_id,
        user_type
)

SELECT
    user_id,
    user_type,
    transaction_count,
    first_purchase,
    last_purchase,
    (CASE
        WHEN user_type = 'VIP' AND transaction_count > 10 THEN 'Possivel Fraude'
        WHEN user_type = 'Regular' AND transaction_count > 5 THEN 'Possivel Fraude'
        ELSE 'Normal'
    END) AS status
FROM
    TransactionCounts
WHERE
    (user_type = 'VIP' AND transaction_count > 10)
    OR (user_type = 'Regular' AND transaction_count > 5);
```

#### Componentes da Query Ajustada:

1. **CTE (Common Table Expression) `RecentTransactions`:**
   - Seleciona as transações que ocorreram nos últimos 10 minutos (`purchase_date >= NOW() - INTERVAL '10 minutes'`).
   - Inclui a coluna `user_type` para segmentar os usuários em diferentes grupos (e.g., 'VIP', 'Regular').

2. **CTE `TransactionCounts`:**
   - Agrupa as transações recentes por `user_id` e `user_type`.
   - Conta o número de transações (`transaction_count`), e determina a data e hora da primeira (`first_purchase`) e última (`last_purchase`) transação no período de 10 minutos.

3. **Consulta Principal:**
   - Seleciona `user_id`, `user_type`, `transaction_count`, `first_purchase`, `last_purchase` e o status da transação.
   - Usa uma expressão `CASE` para verificar se o número de transações no período excede os limites definidos para cada tipo de usuário:
     - Clientes VIP: Limite de 10 transações em 10 minutos.
     - Usuários Regulares: Limite de 5 transações em 10 minutos.
   - Classifica o status como 'Possivel Fraude' se o número de transações exceder os limites definidos, caso contrário, classifica como 'Normal'.

### Benefícios da Query:

- **Precisão Temporal:** Foca nas transações que ocorreram nos últimos 10 minutos, permitindo a detecção de padrões de fraude em tempo real.
- **Relevância:** Reduz o conjunto de dados analisado para apenas as transações recentes, tornando a análise mais eficiente e focada.
- **Personalização:** Ajusta os limites de detecção conforme o perfil de risco de diferentes segmentos de usuários, aumentando a precisão na detecção de fraudes e minimizando interrupções para clientes legítimos.

Implementar esta abordagem ajuda o departamento de prevenção de fraudes a identificar rapidamente atividades suspeitas, protegendo a organização contra perdas financeiras e garantindo a segurança das transações dos clientes.

# 📌 **Detecção de Fraude por Distância Euclidiana em Análise de Padrões de Comportamento**

A query "Detecção de Fraude por Distância Euclidiana em Análise de Padrões de Comportamento" utiliza técnicas de análise de padrões de comportamento para identificar atividades fraudulentas. Em vez de usar apenas médias ou médias móveis, esta abordagem compara as transações atuais com um conjunto de regras derivadas de padrões de comportamento normais. Padrões anômalos, que não se encaixam nesses comportamentos típicos, são marcados como potenciais fraudes. A query utiliza a distância euclidiana entre cada transação e o centroide do comportamento normal do usuário. Utiliza essa distância para classificar transações como 'Possível Fraude' se a distância for significativamente maior do que a média das distâncias (mais de 2 desvios padrão).

#### **Clusters de Comportamento e Centroides:**

- **Clusters de Comportamento:** Agrupamentos de comportamentos de usuários que são similares entre si. Cada cluster representa um padrão de comportamento distinto baseado em características como o valor das transações, a hora do dia em que são realizadas e o dia da semana.
- **Centroides dos Clusters:** Cada cluster tem um centroide, que é um ponto representativo do centro do cluster. Este ponto é calculado como a média dos valores das características de todas as transações no cluster.
- **Centroide do Comportamento:** O centroide de um comportamento é uma média dos valores de comportamento normal de um usuário, como o valor médio das transações, a hora média do dia e o dia médio da semana em que as transações ocorrem.

#### **Importância para o Departamento de Prevenção de Fraudes:**

- **Precisão Melhorada:** Analisar padrões de comportamento permite uma detecção mais precisa de fraudes, reduzindo falsos positivos.
- **Detecção de Fraudes Complexas:** Esta abordagem pode detectar fraudes sofisticadas que não seriam capturadas por técnicas simples de médias ou médias móveis.
- **Ação Proativa:** Permite identificar e agir sobre atividades suspeitas antes que elas causem danos significativos.
- **Adaptação a Novos Padrões:** Ajuda a adaptar o sistema de detecção de fraudes a novos padrões e técnicas usadas por fraudadores.

#### **Construção da Tabela `user_behavior_clusters`:**

A tabela `user_behavior_clusters` é construída através de um processo de clustering, que pode ser feito utilizando algoritmos de machine learning como K-means. O processo envolve:

1. **Coleta de Dados:** Coletar dados de transações históricas dos usuários, incluindo valor das transações, hora do dia e dia da semana.
2. **Pré-processamento:** Limpar e preparar os dados para clustering.
3. **Aplicação do Algoritmo de Clustering:** Aplicar um algoritmo como K-means para agrupar as transações em clusters.
4. **Cálculo dos Centroides:** Calcular o centroide de cada cluster, que representa o comportamento típico dos usuários no cluster.
5. **Armazenamento dos Resultados:** Armazenar os clusters e seus centroides na tabela `user_behavior_clusters`.

**Estrutura da Tabela `user_behavior_clusters`:**

| user_id | cluster_id | centroid_amount | centroid_hour | centroid_day_of_week |
|---------|------------|-----------------|---------------|----------------------|
| 1       | 1          | 500             | 13            | 3                    |
| 1       | 2          | 1500            | 19            | 6                    |
| 2       | 1          | 800             | 10            | 2                    |
| 2       | 2          | 1200            | 18            | 5                    |

#### **Explicação da Query:**

A query a seguir detecta transações que divergem significativamente dos padrões normais de comportamento do usuário. Utiliza um modelo simples de clustering para identificar anomalias. Neste exemplo, transações são comparadas com clusters de comportamento normal previamente calculados.

```sql
-- CTE para calcular os clusters de comportamento normal
WITH NormalBehaviorClusters AS (
    SELECT
        user_id,
        cluster_id,
        centroid_amount,
        centroid_hour,
        centroid_day_of_week
    FROM
        user_behavior_clusters
),
-- CTE para classificar transações como normais ou anômalas
TransactionClassification AS (
    SELECT
        t.user_id,
        t.purchase_date,
        t.amount,
        EXTRACT(HOUR FROM t.purchase_date) AS hour_of_day,
        EXTRACT(DAYOFWEEK FROM t.purchase_date) AS day_of_week,
        nb.cluster_id,
        nb.centroid_amount,
        nb.centroid_hour,
        nb.centroid_day_of_week,
        -- Calcula a distância euclidiana para identificar anomalias
        SQRT(
            POWER(t.amount - nb.centroid_amount, 2) +
            POWER(EXTRACT(HOUR FROM t.purchase_date) - nb.centroid_hour, 2) +
            POWER(EXTRACT(DAYOFWEEK FROM t.purchase_date) - nb.centroid_day_of_week, 2)
        ) AS distance,
        CASE
            WHEN SQRT(
                POWER(t.amount - nb.centroid_amount, 2) +
                POWER(EXTRACT(HOUR FROM t.purchase_date) - nb.centroid_hour, 2) +
                POWER(EXTRACT(DAYOFWEEK FROM t.purchase_date) - nb.centroid_day_of_week, 2)
            ) > (SELECT AVG(SQRT(
                POWER(p.amount - c.centroid_amount, 2) +
                POWER(EXTRACT(HOUR FROM p.purchase_date) - c.centroid_hour, 2) +
                POWER(EXTRACT(DAYOFWEEK FROM p.purchase_date) - c.centroid_day_of_week, 2)
            )) + 2 * STDDEV(SQRT(
                POWER(p.amount - c.centroid_amount, 2) +
                POWER(EXTRACT(HOUR FROM p.purchase_date) - c.centroid_hour, 2) +
                POWER(EXTRACT(DAYOFWEEK FROM p.purchase_date) - c.centroid_day_of_week, 2)
            )) FROM purchases p JOIN user_behavior_clusters c ON p.user_id = c.user_id) THEN 'Possivel Fraude'
            ELSE 'Normal'
        END AS status
    FROM
        purchases t
    JOIN
        NormalBehaviorClusters nb ON t.user_id = nb.user_id
)
SELECT
    user_id,
    purchase_date,
    amount,
    status
FROM
    TransactionClassification
WHERE
    status = 'Possivel Fraude';
```

#### **Componentes da Query:**

1. **CTE `NormalBehaviorClusters`:**
   - Seleciona clusters de comportamento normal de uma tabela `user_behavior_clusters` que contém `user_id`, `cluster_id`, e os centroides dos clusters (`centroid_amount`, `centroid_hour`, `centroid_day_of_week`).

2. **CTE `TransactionClassification`:**
   - Calcula a distância euclidiana entre cada transação e o centroide do comportamento normal do usuário.
   - Utiliza essa distância para classificar transações como 'Possivel Fraude' se a distância for significativamente maior do que a média das distâncias (mais de 2 desvios padrão).

3. **Consulta Principal:**
   - Seleciona transações classificadas como 'Possivel Fraude' com base na análise de padrões de comportamento.

Esta abordagem avançada utiliza análise de padrões de comportamento para detectar fraudes, oferecendo uma detecção mais precisa e robusta contra técnicas sofisticadas de fraude.

# 📌 **Análise de Localização**

A análise de localização é uma técnica utilizada para detectar fraudes examinando os locais geográficos das transações. Ao monitorar a localização das transações de um usuário, é possível identificar padrões suspeitos, como transações ocorrendo em locais geograficamente distantes em um curto espaço de tempo. Isso pode indicar o uso de identidades roubadas ou comprometidas. Esta técnica é vital para o departamento de prevenção de fraudes, pois permite identificar e mitigar fraudes de identidade e outras atividades maliciosas baseadas em localização.

#### **Importância da Segmentação para o Departamento de Prevenção de Fraudes**

A segmentação é essencial na prevenção de fraudes porque permite que as regras e parâmetros de detecção sejam adaptados a diferentes grupos de usuários com base em seus comportamentos e perfis. Isso aumenta a precisão na detecção de fraudes, reduzindo os falsos positivos e concentrando os esforços de investigação onde são mais necessários. Por exemplo, usuários que viajam frequentemente podem ter uma tolerância maior para variações de localização, enquanto usuários com padrões de uso mais estáveis podem ter regras mais rigorosas.

#### **Explicação da Query**

A query a seguir identifica transações que ocorrem em locais geograficamente distantes em um curto espaço de tempo, sugerindo possível uso de identidade roubada. Ela também considera diferentes segmentos de usuários, ajustando os parâmetros de detecção de fraudes para cada grupo.

```sql
WITH LocationAnalysis AS (
    SELECT
        user_id,
        purchase_date,
        location,
        LAG(location) OVER (PARTITION BY user_id ORDER BY purchase_date) AS previous_location,
        LAG(purchase_date) OVER (PARTITION BY user_id ORDER BY purchase_date) AS previous_purchase_date,
        user_type
    FROM
        purchases
)
SELECT
    user_id,
    user_type,
    purchase_date,
    location,
    previous_location,
    previous_purchase_date,
    (CASE
        WHEN previous_location IS NOT NULL
             AND location <> previous_location
             AND ((user_type = 'Frequent Traveler' AND purchase_date - previous_purchase_date < INTERVAL '30 minutes')
                  OR (user_type = 'Regular' AND purchase_date - previous_purchase_date < INTERVAL '1 hour')) THEN 'Possivel Fraude'
        ELSE 'Normal'
    END) AS status
FROM
    LocationAnalysis;
```

#### Componentes da Query:

1. **CTE (Common Table Expression) `LocationAnalysis`:**
   - Seleciona as transações, incluindo `user_id`, `purchase_date`, `location`, e `user_type`.
   - Utiliza a função `LAG` para obter a localização e data da transação anterior do mesmo usuário.

2. **Consulta Principal:**
   - Seleciona `user_id`, `purchase_date`, `location`, `previous_location`, `previous_purchase_date` e o status da transação.
   - Usa uma expressão `CASE` para verificar se a localização atual é diferente da localização anterior e se o intervalo de tempo entre as duas transações é inferior a uma hora.
   - Classifica o status como 'Possivel Fraude' se as condições forem atendidas, caso contrário, classifica como 'Normal'.

### Benefícios da Query:

- **Precisão:** Considera o comportamento individual dos usuários, adaptando os parâmetros de detecção conforme o perfil.
  - *Usuários "Frequent Traveler"*: Esses usuários são mais propensos a realizar transações em diferentes locais com maior frequência devido às viagens.
A condição de tempo é mais restritiva (30 minutos) para capturar mudanças rápidas de localização que ainda são suspeitas, mas considerando a natureza do comportamento desse segmento.
  - *Usuários "Regular"*: Esses usuários tendem a ter padrões de localização mais estáveis. A condição de tempo é mais relaxada (1 hora) para capturar mudanças de localização que são improváveis para usuários com padrões de comportamento mais previsíveis.
- **Relevância:** Aumenta a relevância da detecção, concentrando-se em casos que são mais prováveis de serem fraudulentos.
- **Eficiência:** Reduz falsos positivos ao ajustar as regras para diferentes segmentos, minimizando interrupções para usuários legítimos.

### Contexto e Aplicação

Esta técnica é especialmente útil em detecção de fraudes onde a localização é um fator importante. Por exemplo, uma transação em Nova Iorque seguida de uma transação em Londres dentro de uma hora é improvável para a maioria dos usuários e pode indicar uma atividade fraudulenta.

A segmentação permite que o departamento de prevenção de fraudes adapte as regras de detecção para diferentes grupos de usuários, melhorando a precisão e eficácia na detecção de fraudes. Isso ajuda a proteger a organização contra perdas financeiras e a manter a confiança dos clientes.

# 📌 **Análise de Frequência de Transações**

A análise de frequência de transações é uma técnica utilizada para detectar padrões anômalos de comportamento de usuários, baseando-se na quantidade e frequência das transações realizadas. Esta técnica é fundamental para o departamento de prevenção de fraudes, pois permite identificar atividades suspeitas, como tentativas de exploração de contas comprometidas, onde um fraudador realiza várias transações em rápida sucessão para maximizar o ganho antes que a atividade fraudulenta seja detectada e interrompida.

#### **Importância da Segmentação para o Departamento de Prevenção de Fraudes**

A segmentação é crucial na prevenção de fraudes, pois permite dividir os usuários em diferentes grupos com base em características e comportamentos específicos. Isso possibilita a aplicação de regras de detecção de fraudes mais precisas e adequadas para cada segmento, aumentando a eficácia na identificação de fraudes e reduzindo o número de falsos positivos. Por exemplo, usuários com comportamentos de compra frequentes podem ter limites mais altos, enquanto usuários com padrões de transação mais estáveis podem ter regras mais rigorosas.

#### **Explicação da Query**

A query a seguir identifica usuários que fazem transações em uma frequência anormalmente alta comparada com o seu histórico, marcando transações suspeitas quando o tempo entre elas é muito curto.

```sql
WITH FrequencyAnalysis AS (
    SELECT
        user_id,
        purchase_date,
        EXTRACT(EPOCH FROM purchase_date) - LAG(EXTRACT(EPOCH FROM purchase_date)) OVER (PARTITION BY user_id ORDER BY purchase_date) AS time_diff,
        user_type
    FROM
        purchases
)
SELECT
    user_id,
    user_type,
    purchase_date,
    time_diff,
    (CASE
        WHEN user_type = 'Frequent Buyer' AND time_diff IS NOT NULL AND time_diff < 120 THEN 'Possivel Fraude'
        WHEN user_type = 'Regular' AND time_diff IS NOT NULL AND time_diff < 300 THEN 'Possivel Fraude'
        ELSE 'Normal'
    END) AS status
FROM
    FrequencyAnalysis
WHERE
    time_diff IS NOT NULL;
```

#### Componentes da Query:

1. **CTE (Common Table Expression) `FrequencyAnalysis`:**
   - Seleciona as transações, incluindo `user_id`, `purchase_date`, `user_type` e calcula a diferença de tempo (`time_diff`) entre transações consecutivas usando a função `LAG`.
   - `EXTRACT(EPOCH FROM purchase_date)` converte a data da compra em segundos desde a época Unix (1 de janeiro de 1970).
   - `LAG(EXTRACT(EPOCH FROM purchase_date))` obtém a data da compra anterior em segundos para o mesmo usuário.

2. **Consulta Principal:**
   - Seleciona `user_id`, `user_type`, `purchase_date`, `time_diff` e o status da transação.
   - Usa uma expressão `CASE` para verificar se a diferença de tempo entre transações (`time_diff`) é inferior aos limites definidos para cada tipo de usuário:
     - `Frequent Buyer` (Comprador Frequente): Limite de 120 segundos (2 minutos).
     - `Regular` (Usuário Regular): Limite de 300 segundos (5 minutos).
   - Classifica o status como 'Possivel Fraude' se a diferença de tempo for inferior aos limites, caso contrário, classifica como 'Normal'.

### Benefícios da Query:

- **Precisão Temporal:** Foca nas transações realizadas em intervalos curtos de tempo, permitindo a detecção de padrões de fraude em tempo real.
- **Relevância:** Reduz o conjunto de dados analisado para apenas as transações com diferença de tempo significativa, tornando a análise mais eficiente e focada.
- **Personalização:** Ajusta os limites de detecção conforme o perfil de risco de diferentes segmentos de usuários, aumentando a precisão na detecção de fraudes e minimizando interrupções para clientes legítimos.

### Contexto e Aplicação

Esta técnica é especialmente útil em ambientes onde a frequência de transações pode indicar uma tentativa de fraude. Por exemplo, várias transações consecutivas em um curto período de tempo podem sugerir uma exploração de conta comprometida ou o uso indevido de cartões de crédito.

A segmentação permite que o departamento de prevenção de fraudes adapte as regras de detecção para diferentes grupos de usuários, melhorando a precisão e eficácia na detecção de fraudes. Isso ajuda a proteger a organização contra perdas financeiras e a manter a confiança dos clientes, garantindo uma resposta rápida e eficaz a potenciais ameaças.

# 📌 **Análise de Transações Fora do Horário Comum**

A análise de transações fora do horário comum é uma técnica utilizada para detectar padrões de comportamento anômalos, onde um usuário realiza transações em horários atípicos em comparação com seu comportamento histórico. Esta técnica é essencial para o departamento de prevenção de fraudes, pois permite identificar atividades suspeitas baseadas em anomalias temporais, como transações feitas em horários incomuns para o usuário.

#### **Importância da Segmentação para o Departamento de Prevenção de Fraudes**

A segmentação é crucial para a prevenção de fraudes porque permite que as regras e os parâmetros de detecção sejam adaptados a diferentes grupos de usuários com base em seus comportamentos e perfis. Isso melhora a precisão na detecção de fraudes, reduzindo o número de falsos positivos e focando os esforços de investigação onde são mais necessários. Por exemplo, usuários que tipicamente realizam transações durante o dia podem ter suas transações noturnas marcadas como suspeitas, enquanto usuários noturnos podem ter um padrão diferente de detecção.

#### **Explicação da Query**

A query a seguir identifica transações que ocorrem em horários atípicos para cada usuário, comparando a quantidade de transações realizadas em um determinado horário com a média de transações típicas para esse usuário naquele mesmo horário.

```sql
WITH UserTransactionPatterns AS (
    SELECT
        user_id,
        EXTRACT(HOUR FROM purchase_date) AS hour_of_day,
        COUNT(*) AS transaction_count
    FROM
        purchases
    GROUP BY
        user_id,
        EXTRACT(HOUR FROM purchase_date)
),
UserTypicalHours AS (
    SELECT
        user_id,
        hour_of_day,
        AVG(transaction_count) AS avg_hourly_transactions
    FROM
        UserTransactionPatterns
    GROUP BY
        user_id,
        hour_of_day
)
SELECT
    p.user_id,
    p.purchase_date,
    EXTRACT(HOUR FROM p.purchase_date) AS hour_of_day,
    (CASE
        WHEN uth.avg_hourly_transactions IS NOT NULL
             AND COUNT(p.user_id) OVER (PARTITION BY p.user_id, EXTRACT(HOUR FROM p.purchase_date)) > 2 * uth.avg_hourly_transactions THEN 'Possivel Fraude'
        ELSE 'Normal'
    END) AS status
FROM
    purchases p
LEFT JOIN
    UserTypicalHours uth
ON
    p.user_id = uth.user_id AND EXTRACT(HOUR FROM p.purchase_date) = uth.hour_of_day;
```

#### Componentes da Query:

1. **CTE (Common Table Expression) `UserTransactionPatterns`:**
   - Calcula o número de transações por hora do dia para cada usuário.
   - Usa `EXTRACT(HOUR FROM purchase_date)` para obter a hora da transação.

2. **CTE `UserTypicalHours`:**
   - Calcula a média de transações por hora do dia para cada usuário.
   - Usa `AVG(transaction_count)` para calcular a média de transações por hora.

3. **Consulta Principal:**
   - Seleciona `user_id`, `purchase_date`, `hour_of_day` e o status da transação.
   - Compara o número de transações na hora da compra com a média de transações por hora para o usuário.
   - Classifica a transação como 'Possivel Fraude' se o número de transações for maior que o dobro da média, indicando uma atividade anômala.

### Benefícios:

- **Personalização:** Adapta-se aos padrões de comportamento individual dos usuários, permitindo uma análise mais precisa e personalizada.
- **Eficiência:** Identifica rapidamente transações que ocorrem em horários anômalos, possibilitando uma resposta ágil a potenciais fraudes.
- **Relevância:** Reduz falsos positivos ao considerar o comportamento típico de cada usuário, focando naquilo que realmente é anômalo.

### Contexto e Aplicação

Esta técnica é particularmente útil em ambientes onde o comportamento de transações tem um padrão temporal claro. Por exemplo, se um usuário normalmente faz compras durante o horário comercial e de repente realiza uma transação de alto valor durante a madrugada, isso pode ser um indicativo de fraude.

A segmentação permite que o departamento de prevenção de fraudes adapte as regras de detecção para diferentes grupos de usuários, melhorando a precisão e eficácia na detecção de fraudes. Isso ajuda a proteger a organização contra perdas financeiras e a manter a confiança dos clientes, garantindo uma resposta rápida e eficaz a potenciais ameaças.

# 📌 **Transações em Contas Recém-Criadas**

A query "Transações em Contas Recém-Criadas" identifica transações de alto valor realizadas por contas que foram criadas recentemente, comparando o valor da transação com o dobro da média dos valores gastos por outras contas novas nos primeiros 30 dias após a criação. Este tipo de análise é crucial para o departamento de prevenção de fraudes, pois contas novas são frequentemente usadas por fraudadores para realizar atividades ilícitas antes que possam ser detectadas pelos sistemas de monitoramento.

#### **Importância para o Departamento de Prevenção de Fraudes:**

- **Identificação Precoce de Fraudes:** Contas recém-criadas que fazem transações de alto valor são suspeitas, pois usuários legítimos geralmente demoram algum tempo para realizar grandes compras. Detectar essas transações ajuda a identificar e investigar possíveis fraudes rapidamente.
- **Mitigação de Riscos:** Ao monitorar contas novas, a empresa pode implementar medidas de segurança adicionais, como verificações de identidade, para mitigar o risco de fraude.
- **Proteção Financeira:** A detecção precoce de transações suspeitas em contas novas pode prevenir perdas financeiras significativas. Ao bloquear ou revisar essas transações, a empresa protege seus ativos e reduz o impacto das atividades fraudulentas.
- **Aperfeiçoamento das Políticas de Segurança:** As informações obtidas através desta query podem ajudar a ajustar políticas de segurança e aprimorar os algoritmos de detecção de fraudes para se adaptar às novas técnicas usadas por fraudadores.

#### **Explicação da Query:**

A query a seguir identifica transações de alto valor feitas por contas criadas nos últimos 30 dias, comparando essas transações com o dobro da média dos valores gastos por contas novas nos primeiros 30 dias após a criação.

```sql
-- CTE para selecionar as contas criadas nos últimos 12 meses
WITH NewAccounts AS (
    SELECT
        user_id,
        account_creation_date
    FROM
        users
    WHERE
        account_creation_date >= NOW() - INTERVAL 12 MONTH
),
-- CTE para calcular a média dos valores de transações feitas por contas novas nos primeiros 30 dias
NewCustomerSpending AS (
    SELECT
        AVG(p.amount) AS avg_new_customer_spending
    FROM
        purchases p
    JOIN
        NewAccounts na ON p.user_id = na.user_id
    WHERE
        DATEDIFF(p.purchase_date, na.account_creation_date) <= 30
),
-- CTE para selecionar contas recém-criadas nos últimos 30 dias
RecentAccounts AS (
    SELECT
        user_id,
        account_creation_date
    FROM
        users
    WHERE
        account_creation_date >= NOW() - INTERVAL 30 DAY
)
-- Seleção das transações suspeitas baseadas na média calculada
SELECT
    p.user_id,
    u.account_creation_date,
    p.purchase_date,
    p.amount,
    CASE
        WHEN p.amount > 2 * (SELECT avg_new_customer_spending FROM NewCustomerSpending) THEN 'Possivel Fraude'
        ELSE 'Normal'
    END AS status
FROM
    purchases p
JOIN
    RecentAccounts u ON p.user_id = u.user_id
WHERE
    p.amount > 2 * (SELECT avg_new_customer_spending FROM NewCustomerSpending);
```

#### **Componentes da Query:**

1. **CTE `NewAccounts`:**
   - Seleciona usuários cujas contas foram criadas nos últimos 12 meses.

2. **CTE `NewCustomerSpending`:**
   - Calcula a média dos valores das transações realizadas por essas contas novas nos primeiros 30 dias após a criação da conta.

3. **CTE `RecentAccounts`:**
   - Seleciona usuários cujas contas foram criadas nos últimos 30 dias.

4. **Consulta Principal:**
   - Junta a tabela de transações com a CTE `RecentAccounts` para filtrar apenas transações de contas recém-criadas.
   - Compara o valor da transação com o dobro da média calculada (`2 * avg_new_customer_spending`).
   - Marca transações como 'Possivel Fraude' se o valor for superior ao dobro da média, caso contrário, classifica como 'Normal'.

Esta abordagem ajusta a detecção de fraudes com base em dados reais e recentes, aumentando a precisão e reduzindo falsos positivos, ajudando a equipe de prevenção de fraudes a tomar decisões mais informadas e eficazes.

# 📌 **Análise de Alterações de Informações Pessoais**

A query "Análise de Alterações de Informações Pessoais" identifica mudanças frequentes nas informações pessoais de um usuário, como endereço ou número de telefone, dentro de um curto período de tempo. Este tipo de análise é crucial para o departamento de prevenção de fraudes, pois mudanças frequentes e súbitas nas informações pessoais podem indicar tentativas de fraude, como a criação de identidades falsas ou o comprometimento de uma conta.

#### **Importância para o Departamento de Prevenção de Fraudes:**

- **Identificação de Atividades Suspeitas:** Mudanças frequentes nas informações pessoais de um usuário podem sinalizar atividades fraudulentas, como o uso indevido de contas comprometidas ou a criação de identidades falsas.
- **Mitigação de Riscos:** Detectar essas mudanças rapidamente permite que a equipe de prevenção de fraudes tome medidas imediatas para verificar a autenticidade das alterações e, se necessário, interromper atividades suspeitas.
- **Proteção Financeira:** Ao monitorar e investigar alterações suspeitas nas informações pessoais, a empresa pode prevenir possíveis fraudes e proteger seus ativos financeiros.
- **Aprimoramento das Políticas de Segurança:** As informações obtidas através desta query podem ajudar a ajustar políticas de segurança e melhorar os algoritmos de detecção de fraudes, adaptando-os às novas técnicas usadas por fraudadores.

#### **Explicação da Query:**

A query a seguir identifica usuários que fizeram mudanças frequentes em suas informações pessoais (como endereço ou telefone) nos últimos 30 dias. Se um usuário alterou suas informações mais de 3 vezes dentro desse período, a alteração é marcada como "Possivel Fraude".

```sql
WITH UserInfoChanges AS (
    SELECT
        user_id,
        change_date,
        info_type,
        ROW_NUMBER() OVER (PARTITION BY user_id, info_type ORDER BY change_date DESC) AS change_num
    FROM
        user_info_changes
    WHERE
        change_date >= NOW() - INTERVAL 30 DAY
)
SELECT
    user_id,
    info_type,
    COUNT(*) AS num_changes,
    CASE
        WHEN COUNT(*) > 3 THEN 'Possivel Fraude'
        ELSE 'Normal'
    END AS status
FROM
    UserInfoChanges
GROUP BY
    user_id,
    info_type
HAVING
    COUNT(*) > 3;
```

#### **Componentes da Query:**

1. **CTE `UserInfoChanges`:**
   - Seleciona as mudanças nas informações pessoais (endereço, telefone, etc.) de cada usuário ocorridas nos últimos 30 dias.
   - A função `ROW_NUMBER` é usada para numerar as mudanças ordenadas por data de mudança (`change_date`) para cada tipo de informação (`info_type`) e usuário (`user_id`).

2. **Consulta Principal:**
   - Agrupa as mudanças por `user_id` e `info_type`.
   - Conta o número de mudanças (`num_changes`) feitas por cada usuário em cada tipo de informação.
   - Usa uma expressão `CASE` para marcar as mudanças como 'Possivel Fraude' se o número de mudanças for maior que 3 dentro dos últimos 30 dias, caso contrário, marca como 'Normal'.

Esta abordagem permite ao departamento de prevenção de fraudes identificar e investigar rapidamente atividades suspeitas relacionadas a mudanças frequentes nas informações pessoais dos usuários, ajudando a proteger a empresa contra fraudes e melhorando a segurança dos dados.

# 📌 **Transações Acima de um Limite Específico**

A identificação de transações acima de um limite específico é uma técnica usada para detectar compras ou transferências que excedem um valor pré-determinado, sugerindo uma possível atividade fraudulenta. Esta técnica é crucial para o departamento de prevenção de fraudes, pois permite monitorar transações de alto valor que podem ser indicativas de fraude, especialmente quando o valor é significativamente maior do que as transações típicas do usuário.

#### **Importância da Segmentação para o Departamento de Prevenção de Fraudes**

A segmentação é essencial para a prevenção de fraudes, pois permite dividir os usuários e transações em grupos distintos com base em características e comportamentos específicos. Isso facilita a criação de regras e alertas personalizados para cada segmento, aumentando a precisão na detecção de fraudes. Com a segmentação, é possível ajustar os limites e parâmetros de detecção conforme o perfil de cada grupo, minimizando falsos positivos e concentrando os esforços de investigação onde é mais necessário.

#### **Explicação da Query**

A query a seguir identifica transações que excedem um determinado limite, por exemplo, `R$10.000`. Transações acima desse valor são marcadas como "Possivel Fraude".

```sql
SELECT
    user_id,
    purchase_date,
    amount,
    (CASE
        WHEN amount > 10000 THEN 'Possivel Fraude'
        ELSE 'Normal'
    END) AS status
FROM
    purchases
WHERE
    amount > 10000;
```

#### Componentes da Query:

1. **Seleção de Colunas:**
   - `user_id`: Identificação do usuário que fez a transação.
   - `purchase_date`: Data da transação.
   - `amount`: Valor da transação.

2. **Classificação de Fraude:**
   - Usa uma expressão `CASE` para comparar o valor da transação (`amount`) com o limite especificado (`R$10.000`).
   - Se o valor da transação for maior que `R$10.000`, a transação é classificada como 'Possivel Fraude'.
   - Caso contrário, é classificada como 'Normal'.

3. **Filtro de Transações:**
   - A cláusula `WHERE` garante que apenas as transações acima de `R$10.000` sejam processadas e classificadas.

#### Benefícios:

- **Simplicidade:** Fácil de implementar e entender, ideal para detectar fraudes com base em valores anômalos.
- **Eficiência:** Filtra rapidamente transações de alto valor que podem necessitar de revisão adicional.
- **Personalização:** O limite de valor pode ser ajustado conforme o perfil de risco da organização e dos usuários.

### Contexto e Aplicação

Esta técnica é especialmente útil em cenários onde a maioria das transações possui valores relativamente baixos e, portanto, transações de alto valor são raras e potencialmente suspeitas. Ao monitorar essas transações, o departamento de prevenção de fraudes pode focar em casos que têm uma maior probabilidade de serem fraudulentos, otimizando os recursos e esforços de investigação.

Além disso, a segmentação permite ajustar o limite de valor para diferentes grupos de usuários ou tipos de transações, aumentando a eficácia da detecção de fraudes. Por exemplo, clientes corporativos podem ter limites mais altos, enquanto consumidores individuais podem ter limites mais baixos.

Com essa abordagem, a organização pode proteger melhor seus ativos financeiros e a integridade do sistema, garantindo uma resposta rápida e eficaz a potenciais ameaças.

# 📌 **Taxa de Fraude por Transação**
A "Taxa de Fraude por Transação" é uma métrica que calcula a proporção de transações fraudulentas em relação ao total de transações processadas em um determinado período de tempo. Essa métrica é importante para um negócio por várias razões:

1. **Identificação de Tendências**: Acompanhar a taxa de fraude por transação ao longo do tempo ajuda a identificar tendências e padrões de atividade fraudulenta. Isso permite que a equipe de prevenção de fraudes ajuste suas estratégias de acordo com as mudanças no comportamento dos fraudadores.

2. **Avaliação da Eficácia das Medidas de Prevenção**: Uma taxa de fraude por transação em constante aumento pode indicar que as medidas de prevenção de fraudes atuais não estão sendo eficazes o suficiente. Por outro lado, uma queda na taxa pode sugerir que as medidas implementadas estão sendo bem-sucedidas.

3. **Impacto Financeiro**: Transações fraudulentas podem resultar em perdas financeiras significativas para a empresa. Monitorar a taxa de fraude por transação permite que a empresa avalie o impacto financeiro da fraude e tome medidas para mitigar essas perdas.

Query SQL para acompanhar o histórico da "Taxa de Fraude por Transação" ao longo do tempo:

```sql
SELECT
    DATE_TRUNC('month', transaction_time) AS transaction_month,
    COUNT(CASE WHEN is_fraudulent = 1 THEN 1 END) / COUNT(DISTINCT order_id) AS fraud_transaction_rate
FROM
    transactions_table
GROUP BY
    transaction_month
ORDER BY
    transaction_month;
```

# 📌 **Taxa de Falsos Positivos**
A "Taxa de Falsos Positivos" é uma métrica que mede a proporção de casos identificados como fraudulentos, mas que na verdade são legítimos. Em outras palavras, são transações ou atividades que foram erroneamente classificadas como fraudulentas pela equipe de prevenção de fraudes. A importância dessa métrica em um negócio reside em:

1. **Confiança no Sistema de Detecção de Fraudes**: Uma baixa taxa de falsos positivos indica que o sistema de detecção de fraudes está identificando corretamente as transações suspeitas, sem rotular indevidamente transações legítimas como fraudulentas. Isso aumenta a confiança da empresa e dos clientes no sistema.

2. **Redução de Impactos Negativos**: Falsos positivos podem resultar em inconvenientes para os clientes, como bloqueio indevido de transações legítimas ou contas. Isso pode prejudicar a experiência do cliente e afetar a reputação da empresa. Portanto, manter uma baixa taxa de falsos positivos é essencial para evitar impactos negativos.

3. **Eficiência Operacional**: Reduzir a taxa de falsos positivos ajuda a equipe de prevenção de fraudes a focar seus esforços em casos genuinamente suspeitos, em vez de desperdiçar tempo e recursos investigando falsos alarmes. Isso aumenta a eficiência operacional e permite que a equipe concentre-se nas ameaças reais.

Query SQL para acompanhar o histórico da "Taxa de Falsos Positivos" ao longo do tempo:

```sql
SELECT
    DATE_TRUNC('month', detection_time) AS detection_month,
    COUNT(CASE WHEN is_fraudulent = 1 AND is_legitimate = 1 THEN 1 END) / COUNT(CASE WHEN is_fraudulent = 1 THEN 1 END) AS false_positive_rate
FROM
    fraud_cases_table
GROUP BY
    detection_month
ORDER BY
    detection_month;
```

Esta consulta calculará a taxa de falsos positivos para cada mês, permitindo acompanhar como essa métrica tem variado ao longo do tempo. Isso ajudará a equipe de prevenção de fraudes a avaliar a eficácia das estratégias de detecção de fraudes e fazer ajustes conforme necessário para reduzir os falsos positivos.

# 📌 **Tempo Médio de Detecção de Fraude**

O "Tempo Médio de Detecção de Fraude" é uma métrica que mede o tempo médio decorrido entre a ocorrência de uma transação fraudulenta e sua detecção pela equipe de prevenção de fraudes. Essa métrica é crucial para um negócio porque:

1. **Eficiência na Detecção**: Quanto menor o tempo médio de detecção, mais eficaz é o sistema de prevenção de fraudes em identificar e responder a atividades fraudulentas rapidamente. Isso ajuda a minimizar o impacto financeiro e reputacional da fraude.

2. **Redução de Perdas**: Transações fraudulentas podem resultar em perdas financeiras significativas para a empresa. Uma detecção rápida permite que a empresa tome medidas imediatas para interromper transações fraudulentas em andamento, reduzindo assim o impacto financeiro.

3. **Proteção da Confiança do Cliente**: A detecção rápida de fraudes demonstra compromisso com a segurança do cliente. Isso ajuda a manter a confiança dos clientes na plataforma, garantindo que eles se sintam seguros ao realizar transações.

Query SQL para acompanhar o histórico do "Tempo Médio de Detecção de Fraude" ao longo do tempo:

```sql
SELECT
    DATE_TRUNC('month', detection_time) AS detection_month,
    AVG(detection_time - transaction_time) AS avg_fraud_detection_time
FROM
    fraud_cases_table
GROUP BY
    detection_month
ORDER BY
    detection_month;
```

Esta consulta calculará o tempo médio de detecção de fraude, permitindo acompanhar como essa métrica tem variado ao longo do tempo. Isso fornecerá insights valiosos sobre a eficácia das estratégias de prevenção de fraudes e ajudará a identificar tendências e padrões que possam exigir ajustes nas medidas de segurança.

# 📌 **Valor Médio das Transações Fraudulentas**
O "Valor Médio das Transações Fraudulentas" é uma métrica que calcula a média dos valores das transações identificadas como fraudulentas em um determinado período de tempo. Essa métrica é importante para um negócio por várias razões:

1. **Identificação de Padrões de Fraude**: O conhecimento do valor médio das transações fraudulentas ajuda a identificar padrões de comportamento dos fraudadores. Por exemplo, pode revelar se os fraudadores estão concentrando suas atividades em transações de alto valor ou se estão distribuindo os golpes em transações de valor menor.

2. **Ajuste de Estratégias de Prevenção de Fraudes**: Se houver uma tendência de aumento no valor médio das transações fraudulentas, isso pode indicar a necessidade de ajustar as estratégias de prevenção de fraudes para melhorar a detecção e mitigação dessas transações.

3. **Impacto Financeiro**: Compreender o valor médio das transações fraudulentas é fundamental para avaliar o impacto financeiro da fraude no negócio. Isso permite que a empresa aloque recursos de forma mais eficaz para combater a fraude e reduzir as perdas financeiras.

Query SQL para acompanhar o histórico do "Valor Médio das Transações Fraudulentas" ao longo do tempo:

```sql
SELECT
    DATE_TRUNC('month', transaction_time) AS transaction_month,
    AVG(transaction_amount) AS avg_fraud_transaction_amount
FROM
    transactions_table
WHERE
    is_fraudulent = 1
GROUP BY
    transaction_month
ORDER BY
    transaction_month;
```

Esta consulta calculará o valor médio das transações fraudulentas para cada mês, permitindo acompanhar como esse valor tem variado ao longo do tempo. Isso fornecerá insights valiosos sobre os padrões de gastos dos fraudadores e ajudará na tomada de decisões para combater a fraude de forma eficaz.

# 📌 **Taxa de Recorrência de Fraude por Usuário**
A "Taxa de Recorrência de Fraude por Usuário" é uma métrica que calcula a proporção de vezes que um usuário está envolvido em casos de fraude em relação ao número total de transações realizadas pelo usuário. Essa métrica é importante para um negócio por várias razões:

1. **Identificação de Usuários Suspeitos**: Usuários que têm uma alta taxa de recorrência de fraude podem ser considerados suspeitos de atividades fraudulentas. Identificar esses usuários permite que a equipe de prevenção de fraudes concentre seus esforços de monitoramento e investigação em casos de alto risco.

2. **Prevenção de Fraudes Recorrentes**: Ao detectar usuários com uma taxa de recorrência de fraude significativa, a empresa pode tomar medidas proativas para prevenir fraudes futuras. Isso pode incluir a implementação de verificações adicionais de segurança ou o bloqueio preventivo de transações suspeitas.

3. **Proteção da Integridade da Plataforma**: Reduzir a ocorrência de fraudes recorrentes ajuda a proteger a integridade da plataforma e a manter a confiança dos usuários. Isso é essencial para garantir a longevidade do negócio e o crescimento contínuo.

Query SQL para acompanhar o histórico da "Taxa de Recorrência de Fraude por Usuário" ao longo do tempo:

```sql
SELECT
    DATE_TRUNC('month', transaction_time) AS transaction_month,
    SUM(CASE WHEN is_fraudulent = 1 THEN 1 ELSE 0 END) / COUNT(DISTINCT user_id) AS fraud_recurrence_rate
FROM
    transactions_table
GROUP BY
    transaction_month
ORDER BY
    transaction_month;
```

Esta consulta calculará a taxa de recorrência de fraude por usuário para cada mês, permitindo acompanhar como essa métrica tem variado ao longo do tempo. Isso fornecerá insights valiosos sobre padrões de comportamento suspeitos de usuários e ajudará na identificação e prevenção de fraudes recorrentes.

# 📌 **Tempo Médio para Resolver Casos de Fraude**
O "Tempo Médio para Resolver Casos de Fraude" é uma métrica que calcula o tempo médio necessário para resolver um caso de fraude desde a detecção até a resolução completa. Essa métrica é importante para um negócio por várias razões:

1. **Eficiência Operacional**: Um tempo médio mais curto para resolver casos de fraude indica uma equipe de prevenção de fraudes eficiente e processos bem otimizados. Isso significa que a empresa pode identificar, investigar e resolver casos de fraude de forma rápida e eficaz, minimizando o impacto financeiro e reputacional.

2. **Redução de Perdas Financeiras**: Quanto mais rápido um caso de fraude for resolvido, menor será o tempo em que o fraudador terá para causar danos à empresa. Isso pode resultar em menos perdas financeiras associadas à fraude, pois transações fraudulentas podem ser interrompidas mais rapidamente e medidas corretivas podem ser tomadas imediatamente.

3. **Proteção da Confiança do Cliente**: Resolver casos de fraude rapidamente demonstra compromisso com a segurança dos clientes e ajuda a manter sua confiança na plataforma. Clientes que se sentem seguros ao usar os serviços da empresa são mais propensos a continuar fazendo negócios e recomendando a empresa a outros.

Query SQL para acompanhar o histórico do "Tempo Médio para Resolver Casos de Fraude" ao longo do tempo:

```sql
SELECT
    DATE_TRUNC('month', resolution_time) AS resolution_month,
    AVG(resolution_time - detection_time) AS avg_fraud_resolution_time
FROM
    fraud_cases_table
WHERE
    is_resolved = 1
GROUP BY
    resolution_month
ORDER BY
    resolution_month;
```

Esta consulta calculará o tempo médio para resolver casos de fraude para cada mês em que os casos foram resolvidos, permitindo acompanhar como essa métrica tem variado ao longo do tempo. Isso ajudará a equipe de prevenção de fraudes a avaliar a eficiência de seus processos de resolução e identificar áreas de melhoria, se necessário.

# 📌 **Taxa de Retenção de Clientes após Incidente de Fraude**
A **Taxa de Retenção de Clientes após Incidente de Fraude** é uma métrica que mede a porcentagem de clientes que continuam fazendo negócios com uma empresa após terem sido afetados por um incidente de fraude. Vejamos mais detalhes sobre essa métrica e sua relevância:

- **Definição**: Calcula-se como a proporção de clientes afetados por fraude que permanecem ativos após a resolução do incidente, geralmente medido em um período específico (por exemplo, 12 meses).

- **Importância**:
  - **Fidelidade do Cliente**: Essencial para entender o impacto de incidentes de fraude na lealdade do cliente.
  - **Avaliação de Processos**: Indica a eficácia dos procedimentos de resposta a fraudes e de comunicação com os clientes afetados.
  - **Benchmarking Interno**: Auxilia na comparação de performance entre diferentes períodos ou campanhas de retenção.
  - **Desenvolvimento de Estratégias**: Fornece insights para melhorar a gestão de crises e retenção de clientes.

🔍 **Exemplo de Query SQL**:
A seguir, uma query SQL para calcular a Taxa de Retenção de Clientes após Incidente de Fraude por mês. Supõe-se a existência de uma tabela `incidentes_fraude` com colunas para `id_cliente`, `data_incidente`, e uma tabela `clientes_ativos` com `id_cliente` e `data_atividade`.

```sql
-- Calcula a Taxa de Retenção de Clientes após Incidente de Fraude por mês
WITH incidentes AS (
    SELECT
        id_cliente,
        DATE_FORMAT(data_incidente, '%Y-%m') AS mes_incidente
    FROM
        incidentes_fraude
),
ativos_post_incidente AS (
    SELECT
        i.id_cliente,
        i.mes_incidente,
        COUNT(DISTINCT a.id_cliente) AS clientes_ativos
    FROM
        incidentes i
    JOIN
        clientes_ativos a ON i.id_cliente = a.id_cliente AND DATE_FORMAT(a.data_atividade, '%Y-%m') > i.mes_incidente
    GROUP BY
        i.id_cliente, i.mes_incidente
),
totais_por_mes AS (
    SELECT
        mes_incidente,
        COUNT(DISTINCT id_cliente) AS total_clientes_afetados
    FROM
        incidentes
    GROUP BY
        mes_incidente
)
SELECT
    t.mes_incidente,
    t.total_clientes_afetados,
    COALESCE(a.clientes_ativos, 0) AS clientes_retidos,
    (COALESCE(a.clientes_ativos, 0) / t.total_clientes_afetados) * 100 AS taxa_retensao
FROM
    totais_por_mes t
LEFT JOIN
    ativos_post_incidente a ON t.mes_incidente = a.mes_incidente
ORDER BY
    t.mes_incidente;
```

**Explicação**:
- **incidentes**: Subconsulta para identificar os clientes e o mês de cada incidente de fraude.
- **ativos_post_incidente**: Subconsulta para contar clientes ativos após o incidente, assegurando que a atividade seja posterior ao mês do incidente.
- **totais_por_mes**: Agrupa os incidentes por mês para contar quantos clientes foram afetados.
- **Consulta Principal**: Calcula o total de clientes afetados, quantos foram retidos e a taxa de retenção resultante.

Essa métrica e consulta são vitais para empresas focadas em manter uma base de clientes sólida e confiável, especialmente após situações adversas como fraudes.

# 📌 **Custo Médio por Caso de Fraude Resolvido**
O "Custo Médio por Caso de Fraude Resolvido" é uma métrica financeira utilizada para avaliar a eficiência e o custo de operações relacionadas à resolução de fraudes em uma empresa. Aqui está uma explicação detalhada e a importância dessa métrica:

- **Definição**: Refere-se ao custo médio que uma empresa incurre ao investigar e resolver um único caso de fraude. Inclui custos diretos como mão-de-obra, tecnologia usada na detecção e investigação, e indiretos como perdas devido à fraude.

- **Importância**:
  - **Otimização de Recursos**: Ajuda a determinar se os recursos estão sendo utilizados eficientemente na prevenção e combate à fraude.
  - **Benchmarking**: Permite que a empresa compare sua eficiência com outras do mesmo setor.
  - **Planejamento Financeiro**: Auxilia na alocação de orçamento para áreas de prevenção e detecção de fraudes, visando reduzir custos futuros.
  - **Melhoria Contínua**: Motiva a implementação de melhores práticas e tecnologias para reduzir o custo por caso resolvido.

🔍 **Exemplo de Query SQL**:
A seguir, apresento uma query SQL que calcula o Custo Médio por Caso de Fraude Resolvido mensalmente. Essa consulta pressupõe que você tenha uma tabela chamada `fraudes_resolvidas` com colunas para `data_resolucao`, `custo` e `id_caso`.

```sql
-- Calcula o Custo Médio por Caso de Fraude Resolvido por mês
SELECT
    DATE_FORMAT(data_resolucao, '%Y-%m') AS mes,
    COUNT(id_caso) AS total_casos_resolvidos,
    SUM(custo) AS custo_total,
    SUM(custo) / COUNT(id_caso) AS custo_medio_por_caso
FROM
    fraudes_resolvidas
GROUP BY
    DATE_FORMAT(data_resolucao, '%Y-%m')
ORDER BY
    mes;
```

**Explicação**:
- `DATE_FORMAT(data_resolucao, '%Y-%m')`: Agrupa os dados por mês e ano.
- `COUNT(id_caso)`: Conta o número total de casos de fraude resolvidos em cada mês.
- `SUM(custo)`: Calcula o custo total gasto na resolução dos casos em cada mês.
- `SUM(custo) / COUNT(id_caso)`: Calcula o custo médio por caso resolvido.

Essa métrica e a query SQL são cruciais para empresas que buscam otimizar suas operações e reduzir perdas financeiras devido a fraudes.